# Input and Output in Python

## Libraries and settings

In [ ]:
# Libraries
import os
import re
import json
import folium
import sqlite3
import requests
import pyautogui
import pandas as pd
import matplotlib.pyplot as plt

from zipfile import ZipFile
from bs4 import BeautifulSoup

from PyPDF2 import PdfReader

from reportlab.lib.units import inch
from reportlab.lib.colors import blue
from reportlab.lib.pagesizes import LETTER
from reportlab.pdfgen.canvas import Canvas

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show current working directory
print(os.getcwd())

## Read & write data from/to a database
Most data driven companies store their data in database management system. <b style="color:yellowgreen">SQLite</b> is a lightweight relational database management system (RDBMS). With python you can connect to a SQLite DB and make requests using SQL.

### Write data to a database

In [ ]:
# Create e new db
conn = sqlite3.connect('data/example_sqlite.db')

# Close connection to db
conn.close()

In [ ]:
# Open connection to db
conn = sqlite3.connect('data/example_sqlite.db')

# Define variables and data types for the (empty) table
conn.execute('''CREATE TABLE IF NOT EXISTS COMPANY
             (ID INT PRIMARY KEY     NOT NULL,
             NAME           TEXT    NOT NULL,
             AGE            INT     NOT NULL,
             CITY        CHAR(50),
             SALARY         REAL);''')

# Read data from a file and write to data frame
data = pd.read_excel("data/db_data.xlsx", sheet_name = "Sheet1")
print(data)

# Write data to the data base table named 'COMPANY'
data.to_sql('COMPANY', conn, if_exists='replace')

# Commit the changes to the table
conn.commit()

# Close connection to db
conn.close()

### Query the database using SQL and write result to a pandas data frame

In [ ]:
# Connection to db
conn = sqlite3.connect("data/example_sqlite.db")

# Read data
df_sub = pd.read_sql('''SELECT * 
                        FROM COMPANY 
                        WHERE AGE <= 26''', 
                     con=conn,
                     index_col=['index'])
print(df_sub)
    
# Close connection to db
conn.close()

## Read & write data from/to files

### Common data/file formats

Data formats in information technology may refer to:

- Data type, constraint placed upon the interpretation of data in a type system
- Signal (electrical engineering), a format for signal data used in signal processing
- Recording format, a format for encoding data for storage on a storage medium
- <b style="color:yellowgreen">File format, a format for encoding data for storage in a computer file</b>
- Container format (digital), a format for encoding data for storage by means of a standardized audio/video codecs file format
- Content format, a format for representing media content as data
- Audio format, a format for encoded sound data
- Video format, a format for encoded video data

Wikipedia: https://en.wikipedia.org/wiki/Data_format

This section provides common <b style="color:yellowgreen">file formats</b> a data scientist or a data engineer must be aware of. Later, we’ll see how to read these file formats in Python.

List with common file formats explained in this notebook:
- CSV
- TXT
- JSON
- XML
- HTML
- ZIP
- XLSX
- PDF
- Image files (e.g. JPEG)

### CSV (comma separated value)

- A comma-separated values (CSV) file is a delimited text file.
- Each line of the file is a data record.
- Each record consists of one or more fields, separated by a separator (default = comma).
- The use of the comma as a field separator is the source of the name for this file format.
- The seperator can also be user-defined, e.g. you can also use a semicolon instead of a comma.
- A CSV file typically stores tabular data (numbers and text).

In [ ]:
# Read data from .csv-file using pandas (here the separator is a semicolon)
data = pd.read_csv("data/example.csv", sep=";")

# Print the header info of data (first five rows)
print(data.head(5))

# Write data to csv
data.to_csv("data/example_write.csv", sep=";")

### TXT (plain text)

- In Plain Text file format, everything is written in plain text
- Usually, this text is in unstructured form and there is no meta-data associated with it
- The TXT file format can easily be read by any program

In [ ]:
# Open a connection to the text-file
text_file = open("data/example.txt", 
                 "r", 
                 encoding='utf-8')

# Read data from .txt file
lines = text_file.read()

# Show type
print(type(lines))

# Print the data
print(lines)

In [ ]:
# Write data to a .txt file
lines = ['Dorothy lived in the midst of the great Kansas prairies', 
         'with Uncle Henry, who was a farmer ...']

with open('data/example_write.txt', 'w') as f:
    f.writelines(lines)
    
# Check whether file exists
files = [f for f in os.listdir('.') if re.match('data/example_write.txt', f)]
print(files)

### JSON (JavaScript Object Notation)

- JSON is a syntax for storing and exchanging data
- JSON is text, written with JavaScript object notation

In [ ]:
# Read data
with open('data/example.json', 'r') as f:
    data = json.load(f)
print(data)

# Read data to a data frame using the pandas library
data = pd.read_json("data/example.json")

# Print the data
print('\n',data)

In [ ]:
# Write data to .json 
data.to_json('data/example_write.json')

# Check whether the file exists
files = [f for f in os.listdir('.') if re.match('data/example_write.json', f)]
print(files)

### XML (extensible markup language)

- XML stands for extensible Markup Language
- XML is a markup language much like HTML
- XML was designed to store and transport data
- XML was designed to be self-descriptive
- XML is a W3C Recommendation

In [ ]:
# First option: reading the xml file with BeautifulSoup
bs = BeautifulSoup(open('data/example.xml'), 'html.parser')
print(bs.prettify())

# Write data to xml with BeautifulSoup
f = open('data/example_write.xml', "w")
f.write(bs.prettify())
f.close()

In [ ]:
# Second option: reading the xml file using .read_xml() from pandas
data = pd.read_xml("data/example.xml")

# Write data to .xml
data.to_xml('data/example_write.xml')

# Check whether file exists
files = [f for f in os.listdir('.') if re.match('data/example_write.xml', f)]
print('\n', data[['name', 'price', 'calories']])

### HTML (hyper text markup language)

- HTML stands for Hyper Text Markup Language
- HTML is the standard markup language for creating Web pages
- HTML describes the structure of a Web page
- HTML consists of a series of elements
- HTML elements tell the browser how to display the content
- HTML elements label pieces of content such as "this is a heading", "this is a paragraph", "this is a link", etc.

In [ ]:
# Read data from .html
filename = 'data/example.html'
html = open(filename, "r").read()
print(html)

In [ ]:
# Write data to .html (taking the html-file from above)
with open('data/example_write.html', 'w') as f:
    f.writelines(html)
    
# Check whether file exists
files = [f for f in os.listdir('.') if re.match('data/example_write.html', f)]
print(files)

### ZIP (archive file format)

- ZIP is an archive file format that supports lossless data compression
- A ZIP file may contain one or more files or directories that may have been compressed
- The ZIP file format permits a number of compression algorithms, though DEFLATE is the most common
- The name "zip" (meaning "move at high speed") was suggested by R. Mahoney
- They wanted to imply that their product would be faster than ARC and other compression formats of the time

In [ ]:
# Pandas supports zip file reads
data = pd.read_csv("data/archive.zip", sep=";")
data.head(5)

In [ ]:
# Create an empty Zip-archive
zipObj = ZipFile('data/example_write.zip', 'w')

# Add selected files to the zip archive
zipObj.write('data/example.csv')
zipObj.write('data/example.html')
zipObj.write('data/example.json')

# Close the Zip-archive
zipObj.close()

# Check whether zip-file exists
files = [f for f in os.listdir('.') if re.match('data/example_write.zip', f)]
print(files)

### XLSX (Microsoft Excel Open XML file format)

- It is an XML-based file format created by Microsoft Excel 
- The XLSX format was introduced with Microsoft Office 2007
- In XLSX data is organized under the cells and columns in a sheet
- Each XLSX file may contain one or more sheets
- A single workbook can contain multiple sheets

In [ ]:
# Read data from an example .xlsx-file
data = pd.read_excel("data/example.xlsx", sheet_name = "sheet1")

# Print the data 
data.head(5)

In [ ]:
# Write data to xlsx
data.to_excel('data/example_write.xlsx', sheet_name = "sheet1")

# Check whether file exists
files = [f for f in os.listdir('.') if re.match('data/example_write.xlsx', f)]
print(files)

### PDF (portable document format)

- PDF is a file format developed by Adobe in the 1990s to present documents, including text formatting and images, in a manner independent of application software, hardware, and operating systems
- Based on the PostScript language, each PDF file encapsulates a complete description of a fixed-layout flat document, including the text, fonts, vector graphics, raster images and other information needed to display it

In [ ]:
# Reading metadata
reader = PdfReader("data/example.pdf")
meta = reader.metadata
print(len(reader.pages))

# All of the following could be None!
print(meta.author)
print(meta.creator)
print(meta.producer)
print(meta.subject)

# Extract text
page = reader.pages[0]
print('\n')
print(page.extract_text())

# Number of pages
print('\n')
print(f'Number of pages in PDF: {len(reader.pages)}')


In [ ]:
# Create a canvas
canvas = Canvas("data/example_write.pdf", pagesize = LETTER)

# Set font to Times New Roman with 36-point size
canvas.setFont("Times-Roman", 36)

# Draw blue text one inch from the left and ten inches from the bottom
canvas.setFillColor(blue)
canvas.drawString(1 * inch, 10 * inch, "This is a PDF file ...")

# Save the PDF file
canvas.save()

# Check whether file exists
files = [f for f in os.listdir('.') if re.match('data/example_write.pdf', f)]
print(files)

### Image file formats

- Image files consists of pixels and meta-data associated with it
- Usual image files are 3-dimensional, having RGB values
- Image files can also be 2-dimensional (grayscale) or 4-dimensional (having intensity)
- Each image consists one or more frames of pixels
- Each frame is made up of two-dimensional array of pixel values
- Pixel values can be of any intensity 
- Meta-data associated with an image, can be an image type (.png) or pixel dimensions
- The different formats (JPEG, PNG, TIFF, GIF, ...) are used to organize and store digital images in a different way

In [ ]:
# Read image
image = plt.imread('data/example.jpeg')

# Plot image
plt.figure(figsize=(6,4))
plt.imshow(image)

In [ ]:
# Take a screenshot of your computer
pic = pyautogui.screenshot()

# Save the screenshot
pic.save('data/example_write.jpeg')

# Read the image
image = plt.imread('data/example_write.jpeg')

# Plot the image
plt.figure(figsize=(6,4))
plt.imshow(image)

### Spatial data

#### Polygon map of municipalities in the canton of Zürich

In [ ]:
# Read spatial data
polys = 'data/GEN_A4_GEMEINDEN_2019_epsg4326.json'

# Initialisierung der Map
m = folium.Map(location=[47.44, 8.65], zoom_start=10)

# Map settings
folium.Choropleth(
    geo_data=polys,
    name='polys',
    fill_color='greenyellow'
).add_to(m)

folium.LayerControl().add_to(m)

# Plot map
m

#### Overpass turbo query to get all available restaurants in a defined city

In [ ]:
# Overpass API URL
url = 'http://overpass-api.de/api/interpreter'

# Overpass turbo query
query = f"""
        [out:json];
        area[name="Winterthur"];
        node["amenity"="restaurant"](area);
        out;"""

# Web API request
r = requests.get(url, params={'data': query})
data = r.json()['elements']

# Save data to file
with open('data/restaurants.json', 'w') as json_file:
    json.dump(data, json_file)

# Store data in data frame
df = pd.json_normalize(data)
df.head()

#### Plot restaurants on map

In [ ]:
# Subset of gas stations by brand
locations = df[["lat", "lon", "tags.name"]]
print(locations.head(5))

# Create map
map = folium.Map(location=[locations.lat.mean(), 
                           locations.lon.mean()], 
                 zoom_start=14, 
                 control_scale=True)

# Add maker symbols
for index, location_info in locations.iterrows():
    folium.Marker([location_info["lat"], 
                   location_info["lon"]], 
                  popup=location_info["tags.name"]).add_to(map)

# Plot map
map

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')